In [5]:
import requests
import pandas as pd
import json
import time
import aiohttp
import asyncio
import signal
import nest_asyncio
nest_asyncio.apply()

In [101]:
# Make an API request to get player IDs from the leaderboard
players = []
url = f"https://omeda.city/players.json"
pages = [i for i in range(1, 251)]

base_params = {
    "filter[include_inactive]" : 0,
    "filter[include_unranked]" : 0
}
for page in pages:
    params = base_params.copy()
    params["page"] = page
    try:
        response = requests.get(url, params=params, timeout=10)  
        response.raise_for_status()  
        data = response.json()
        df = pd.DataFrame(data)
        df1 = df.filter(items = ['id'], axis = 1)
        players.append(df1)
    except requests.exceptions.RequestException as e:
        print(f"API request error: {e}")
playerlist = pd.concat(players, ignore_index=True)
playerlist.to_csv(r'F:\Downloads\playerids.txt', index=False, header=False)

In [4]:
params = {
    "per_page": 10,                 
    "filter[game_mode]": "ranked"    
}
file = r'F:\Downloads\playerids.txt'
progress = 'F:/Downloads/progress.json'
output = 'F:/Downloads/results.json'
max_retries = 3

In [6]:
# Take retrieved player IDs for further match lookup
def load_ids():
    with open(file, "r") as f:
        return [line.strip() for line in f.readlines()]

In [8]:
# Take already processed IDs from the progress list
def load_progress():
    try:
        with open(progress, "r") as f:
            return json.load(f)
    except FileNotFoundError:
        return {"processed_ids": []} # 

In [10]:
# Append newly processed IDs to the progress list
def save_progress(processed_ids):
    with open(progress, "w") as f:
        json.dump({"processed_ids": processed_ids}, f)

In [12]:
# Save API responses to the result file
def save_results(data):
    try:
        with open(output, "r") as f:
            results = json.load(f)
    except FileNotFoundError:
        results = []

    results.append(data)
    with open(output, "w") as f:
        json.dump(results, f, indent=4)

In [14]:
# Track the number of processed IDs
def check_progress():
    try:
        with open(progress, "r", encoding="utf-8") as f:
            progress_data = json.load(f)
            return set(progress_data.get("processed_ids", [])) 
    except (FileNotFoundError, json.JSONDecodeError):
        return set()

In [16]:
# Calculate the overall progress percentage
def calculate_progress():
    all_ids = load_ids()
    processed_ids = check_progress()

    total = len(all_ids)
    processed = len(processed_ids)

    if total == 0:
        return "⚠️ No IDs in playerids.txt!"
    
    progress_percent = (processed / total) * 100
    return f"📊 Progress: {processed}/{total} processed ({progress_percent:.2f}%)"

In [18]:
# Make asynchronous API requests to fetch player match data
async def fetch_data(session, url, player_id, processed_ids):
    retries = 0
    while retries < max_retries:
        try:
            async with session.get(url, params=params, timeout=60) as response:
                response.raise_for_status() 
                data = await response.json()
    
                if data:
                    save_results(data)
                    processed_ids.add(player_id)
                    save_progress(list(processed_ids))
                return  
        except asyncio.TimeoutError:
            print(f"⚠ Timeout for player_id: {player_id}. Retrying...")
            retries += 1
            await asyncio.sleep(10)
        except asyncio.CancelledError:
            print(f"\n⏳ Cancelling request for ID {player_id}...")
            raise
        except aiohttp.ClientError as e:
            print(f"Error: {e}. Retry ({retries + 1}/{max_retries})...")
            retries += 1
            await asyncio.sleep(5)  
    if retries == max_retries:
        print(f"⚠ Skip ID {player_id} after {max_retries} failed attempts.")

async def process_ids():
    all_ids = load_ids()
    progress = load_progress()
    processed_ids = set(progress["processed_ids"])
    tasks = set()
    async with aiohttp.ClientSession() as session:
        try:
            for id_ in all_ids:
                if id_ in processed_ids:
                    continue
        
                print(f"\r{calculate_progress()}", end="", flush=True)
        
                player_id = id_.strip()
                url = f"https://omeda.city/players/{player_id}/matches.json"
        
                
                task = asyncio.create_task(fetch_data(session, url, player_id, processed_ids))
                tasks.add(task)
        
                # the maximum concurrency of API requests
                if len(tasks) >= 3:
                    done, pending = await asyncio.wait(tasks, return_when=asyncio.FIRST_COMPLETED)
                    tasks.difference_update(done)
        
           
            if tasks:
                await asyncio.gather(*tasks)
            
        except asyncio.CancelledError:
            print("\n⏳ Cancelling all pending requests...")
            for task in tasks:
                task.cancel()  
            await asyncio.gather(*tasks, return_exceptions=True)  
            print("⚠ Processing stopped.")
            raise  
    print("\n✅ All IDs processed!")

In [ ]:
try:
    await process_ids()
    print("\n✅ All IDs processed!")
except KeyboardInterrupt:
    print("\n⚠ Request stopped. Waiting for the last request to finish...")
except asyncio.CancelledError:
    print("\n⏹ All async tasks cancelled.")

In [41]:
pd.set_option('display.max.columns', 235)

In [22]:
# Step 1: Build a base table with all results by expanding columns to include match and player details
with open(output, "r", encoding="utf-8") as file:
    matches = json.load(file)
df = pd.json_normalize(matches, record_path='matches')
df.rename(columns={"id": "match_id"}, inplace=True)
df = pd.DataFrame(df)
df_expanded = df.explode("players", ignore_index=True)
df_players = pd.json_normalize(df_expanded["players"])
df_players["match_id"] = df_expanded["match_id"]
df_main = df.drop(columns=["players"])

In [24]:
df_main

,match_id,start_time,end_time,game_duration,game_region,region,winning_team,game_mode
0,7069615d-081d-4fb5-9d1e-1547510fcf06,2025-02-11T06:39:47.000Z,2025-02-11T07:16:46.000Z,2217,naeast,NA_EAST,dusk,ranked
1,2e4a3f88-4c0b-4929-ac82-dd09c558525c,2025-02-11T05:50:18.000Z,2025-02-11T06:27:17.000Z,2217,naeast,NA_EAST,dusk,ranked
2,cf90be2d-7101-44f5-ab8a-0b2fbec85d48,2025-02-11T01:25:21.000Z,2025-02-11T02:12:52.000Z,2850,naeast,NA_EAST,dawn,ranked
3,8cba9b86-3b4d-4da5-ae74-b5ddd61e3ecf,2025-02-11T00:27:02.000Z,2025-02-11T01:05:52.000Z,2329,naeast,NA_EAST,dusk,ranked
4,3723d80f-d802-4530-b84b-d7c01c57e7b5,2025-02-10T23:44:58.000Z,2025-02-11T00:21:28.000Z,2189,naeast,NA_EAST,dusk,ranked
...,...,...,...,...,...,...,...,...
38983,71a28bdd-49f3-401a-a3a7-c5a2a5e6bf3f,2025-01-22T01:41:15.000Z,2025-01-22T02:18:39.000Z,2243,naeast,NA_EAST,dawn,ranked
38984,d2d78763-f1d0-4fa6-8731-6c0c40e1515b,2025-01-22T00:59:41.000Z,2025-01-22T01:32:45.000Z,1983,naeast,NA_EAST,dawn,ranked
38985,a5cb3650-d0cc-4132-951f-ce788e8e2183,2025-01-18T23:49:47.000Z,2025-01-19T00:11:27.000Z,1299,naeast,NA_EAST,dusk,ranked
38986,b4aebd9c-2026-4608-a552-548e4996807a,2024-11-13T00:29:34.000Z,2024-11-13T01:02:56.000Z,2000,naeast,NA_EAST,dawn,ranked


In [26]:
# Step 2: Add platform labels (PC and Console) and remove duplicate matches
df2 = pd.DataFrame(df_players)
df2["display_name"] = df2["display_name"].replace("🎮", "CONSOLE", regex=True)
df3 = df2.filter(items = ['match_id','team', 'display_name'], axis = 1)
df4 = df3.drop_duplicates()

In [28]:
# Step 3: Merge match data with player data
merged_df = df_main.merge(df4, on="match_id", how="inner")
merged_df["result"] = merged_df.apply(lambda row: "Win" if row["team"] == row["winning_team"] else "Lose", axis=1)
merged_df["platform"] = merged_df["display_name"].apply(lambda x: "Console" if "console" in x.lower() else "PC")
merged_df["platform_count"] = merged_df["platform"].apply(lambda x: 1 if "console" in x.lower() else 0)
raw_data = merged_df.filter(items = ['match_id','team', 'platform', 'result', 'game_region', 'game_duration', 'platform_count', 'display_name'], axis = 1)
raw_data1 = raw_data.drop_duplicates()
raw_data1.to_csv(r'F:\Downloads\clean_data_leaderboard.csv')

In [30]:
raw_data1

,match_id,team,platform,result,game_region,game_duration,platform_count,display_name
0,7069615d-081d-4fb5-9d1e-1547510fcf06,dawn,PC,Lose,naeast,2217,0,MrCurser
1,7069615d-081d-4fb5-9d1e-1547510fcf06,dawn,PC,Lose,naeast,2217,0,BadYoshi23
2,7069615d-081d-4fb5-9d1e-1547510fcf06,dawn,PC,Lose,naeast,2217,0,Maguli's lil Bro
3,7069615d-081d-4fb5-9d1e-1547510fcf06,dawn,PC,Lose,naeast,2217,0,ninjaweenie
4,7069615d-081d-4fb5-9d1e-1547510fcf06,dawn,PC,Lose,naeast,2217,0,IBRACREAMOVIC
...,...,...,...,...,...,...,...,...
390907,92002f2e-b682-45c6-b2ed-47bea6f75539,dusk,PC,Lose,naeast,1120,0,Mercenary_Wraith
390908,92002f2e-b682-45c6-b2ed-47bea6f75539,dusk,PC,Lose,naeast,1120,0,Bechnasty89 T_TV
390909,92002f2e-b682-45c6-b2ed-47bea6f75539,dusk,Console,Lose,naeast,1120,1,CONSOLE user-7166f84c
390910,92002f2e-b682-45c6-b2ed-47bea6f75539,dusk,Console,Lose,naeast,1120,1,CONSOLE user-ae0fb1df


In [7]:
# Step 4: Group data by match IDs
raw_data1 = pd.read_csv(r'F:\Downloads\Docs\clean_data_leaderboard.csv', index_col=0)
raw_data1["wins"] = raw_data1["result"].apply(lambda x: 1 if x == "Win" else 0)
df5 = raw_data1.groupby(['match_id', 'team', 'result', 'game_region', 'game_duration', 'wins']).sum(numeric_only = True)
df6 = pd.DataFrame(df5)
df6 = df6.reset_index()

In [21]:
df6

,match_id,team,result,game_region,game_duration,wins,platform_count
0,00077b93-fafe-4b7d-a41e-2172819f9088,dawn,Lose,europe,2131,0,2
1,00077b93-fafe-4b7d-a41e-2172819f9088,dusk,Win,europe,2131,1,3
2,0009ea96-9b4b-4a42-80e4-a7b348cd52df,dawn,Win,europe,1982,1,1
3,0009ea96-9b4b-4a42-80e4-a7b348cd52df,dusk,Lose,europe,1982,0,3
4,000ae4b1-2bf1-4d81-a88e-e4a821ec2672,dawn,Lose,europe,1477,0,4
...,...,...,...,...,...,...,...
46129,fff7c5e7-d4a6-4cca-9123-16e1c28a1627,dusk,Win,naeast,1682,1,2
46130,fff7fc8c-3578-4f82-a6e0-8c6eaddfdf76,dawn,Win,europe,1624,1,2
46131,fff7fc8c-3578-4f82-a6e0-8c6eaddfdf76,dusk,Lose,europe,1624,0,2
46132,fffd5199-7166-4190-a8b6-d1dfe3146b2d,dawn,Lose,naeast,1976,0,1


In [9]:
# Calculate win rates for teams with varying numbers of console players
platform_winrate = df6.groupby("platform_count").agg(
    win_rate_absolute=("wins", lambda x: x.sum() / df6.loc[x.index, "result"].count())
).round(2)
df7 = df6.merge(platform_winrate, on="platform_count", how="left")

In [11]:
platform_winrate

,win_rate_absolute
platform_count,
0,0.52
1,0.51
2,0.50
3,0.49
4,0.47
5,0.48


In [13]:
# Calculate the difference in the number of PC and console players for each match
pivot_df = df7.pivot(index="match_id", columns="team", values="platform_count")
pivot_df["platform_delta_dawn"] = pivot_df["dawn"] - pivot_df["dusk"]
pivot_df["platform_delta_dusk"] = pivot_df["dusk"] - pivot_df["dawn"]
df8 = df7.merge(pivot_df[["platform_delta_dusk", "platform_delta_dawn"]], on="match_id", how="left")
df8["platform_delta"] = df8.apply(lambda row: row["platform_delta_dawn"] if row["team"] == "dawn" else row["platform_delta_dusk"], axis=1)
df8.drop(columns=["platform_delta_dawn", "platform_delta_dusk"], inplace=True)

In [15]:
# Calculate win rates grouped by the difference in player count
platform_delta_winrate = df8.groupby("platform_delta").agg(
    win_rate_delta=("wins", lambda x: x.sum() / df8.loc[x.index, "result"].count())
).round(2)
df9 = df8.merge(platform_delta_winrate, on="platform_delta", how="left")

In [19]:
platform_delta_winrate 
# Data indicates lower win rates when console players are overrepresented in one team

,win_rate_delta
platform_delta,
-5,0.62
-4,0.56
-3,0.56
-2,0.53
-1,0.51
0,0.50
1,0.49
2,0.47
3,0.44


In [25]:
# The number of results (matches) for every difference in players
df9.groupby('platform_delta').agg({'wins':['sum'],'result':['count']})

,wins,result
,sum,count
platform_delta,,
-5,39,63
-4,303,540
-3,1102,1973
-2,2726,5158
-1,4734,9194
0,6139,12278
1,4460,9194
2,2432,5158
